In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
from datetime import datetime
import itertools
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import plot_importance, plot_tree


In [ ]:
# Reading the data from the file
df = pd.read_csv('result_cc_consumption_1.csv')

#Converting uch_level_g column to category so that, in next step we can encode it. ML algos want numeric values.
df["UCH_LEVEL_G"] = df["UCH_LEVEL_G"].astype('category')
df["UCH_LEVEL_G_CAT"] = df["UCH_LEVEL_G"].cat.codes

#df["UCH_LEVEL_G"] = ['div_' + str(i) for i in df['UCH_LEVEL_G']]
#df['BASECODE'] = ['item_' + str(i) for i in df['BASECODE']]
#df['time_series'] = df[['UCH_LEVEL_G', 'BASECODE']].apply(lambda x: '_'.join(x), axis=1)
#df.drop(['UCH_LEVEL_G', 'BASECODE'], axis=1, inplace=True)

#Converting column names in lower case
df.columns = [c.lower() for c in df.columns]

#Converting fscl_wk_end_dt column to datetime format
df['fscl_wk_end_dt'] = pd.to_datetime(df['fscl_wk_end_dt'])

# Adding a new feature column promo_ind
df.loc[df.planned_ptc * .95 > df.suggested_ptc, 'promo_ind'] = 0
df.loc[~(df.planned_ptc * .95 > df.suggested_ptc), 'promo_ind'] = 1
df.promo_ind = df.promo_ind.astype(int)

cust_basecode_max_sales_dt_df = df.groupby(['uch_level_g_cat','basecode'])['fscl_wk_end_dt'].agg(["max"]).reset_index()
cust_basecode_max_sales_dt_df['max_sales_year'] = pd.DatetimeIndex(cust_basecode_max_sales_dt_df['max']).year

now = datetime.now()
year = now.year

# Adding a new feature column promo_ind
cust_basecode_max_sales_dt_df.loc[pd.DatetimeIndex(cust_basecode_max_sales_dt_df['max']).year >= year - 1, 'recent_year_flag'] = 1
cust_basecode_max_sales_dt_df.loc[~(pd.DatetimeIndex(cust_basecode_max_sales_dt_df['max']).year >= year - 1), 'recent_year_flag'] = 0
cust_basecode_max_sales_dt_df.recent_year_flag = cust_basecode_max_sales_dt_df.recent_year_flag.astype(int)
df.set_index('fscl_wk_end_dt',inplace=True)
cust_basecode_max_sales_dt_df

# Getting unique combination of div and basecode
# This dataset will be used to loop thru to run the model fir each combination, provided, they have sufficient number of records to process
unique_comb_df = df[['uch_level_g_cat','basecode']].loc[:,['uch_level_g_cat','basecode']].reset_index().drop('fscl_wk_end_dt',axis=1).drop_duplicates().dropna()
unique_comb_df['unique_id'] = unique_comb_df.uch_level_g_cat.astype(str) + '_' + unique_comb_df.basecode.astype(str)
unique_comb_df.sort_values(by='uch_level_g_cat', ascending=True, inplace=True )
unique_comb_df

#Dropping no sales records, as souce has zero scan logic, the source data set comes with zero sales records.
df = df.dropna()

# extract features from date
#df['month'] = [i.month for i in df['fscl_wk_end_dt']]
#df['year'] = [i.year for i in df['fscl_wk_end_dt']]
#df['day_of_year'] = [i.dayofyear for i in df['fscl_wk_end_dt']]
df

,uch_level_g,basecode,suggested_ptc,planned_ptc,holiady_ind,curr_yr_sales_dollars,curr_yr_sales_units,ptc,selling_store_count,actual_str_cnt,selling_store_ratio,promotion_depth,lag_1_promotion_depth,lag_1_ptc,uch_level_g_cat,promo_ind
fscl_wk_end_dt,,,,,,,,,,,,,,,,
2017-01-14,CA4000301,681000111700,4.32,4.99,0,19889.93,5326.0,3.734497,440,588,0.748299,1.155093,1.155093,4.99,0,0
2017-01-14,CA4000301,681000112500,4.32,4.99,0,76399.73,20544.0,3.718834,455,588,0.773810,1.155093,1.155093,4.99,0,0
2017-01-14,CA4000301,681000145500,3.04,4.49,0,31943.97,14343.0,2.227147,429,588,0.729592,1.476974,0.654605,1.99,0,0
2017-01-14,CA4000301,681000145900,3.04,4.49,0,23270.63,9537.0,2.440037,419,588,0.712585,1.476974,0.654605,1.99,0,0
2017-01-14,CA4000301,681000146500,3.04,4.49,0,22298.79,9607.0,2.321098,358,588,0.608844,1.476974,0.654605,1.99,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-15,CA4001319,681009006400,5.76,5.99,0,21.89,11.0,1.990000,1,963,0.001038,1.039931,1.039931,5.99,24,1
2021-05-15,CA4001319,681009006400,5.76,6.46,0,83.27,13.0,6.405385,10,963,0.010384,1.121528,0.887153,5.11,24,0
2021-05-15,CA4001319,681009006600,4.32,5.40,0,100.13,17.0,5.890000,12,963,0.012461,1.250000,1.233796,5.33,24,0


In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE()
X_embedded = tsne.fit_transform(df[['holiady_ind','selling_store_ratio','promotion_depth','lag_1_promotion_depth','lag_1_ptc','promo_ind']])

In [ ]:
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=df.curr_yr_sales_units, legend='full')

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning:

Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.



ValueError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
!pip install mlxtend  

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

In [ ]:
from mlxtend.feature_selection import ColumnSelector

col_selector = ColumnSelector(cols=(0, 1))
# col_selector.fit(X) # optional, does not do anything
col_selector.transform(X).shape

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



(150, 2)

In [ ]:
import pandas as pd

iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [ ]:
col_selector = ColumnSelector(cols=("sepal length (cm)", "sepal width (cm)"))
col_selector.transform(iris_df).shape

(150, 2)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


pipe = make_pipeline(StandardScaler(),
                     ColumnSelector(cols=(0, 1)),
                     KNeighborsClassifier())

pipe.fit(X, y)
pipe.score(X, y)

0.84

In [ ]:
df.columns

Index(['uch_level_g', 'basecode', 'suggested_ptc', 'planned_ptc',
       'holiady_ind', 'curr_yr_sales_dollars', 'curr_yr_sales_units', 'ptc',
       'selling_store_count', 'actual_str_cnt', 'selling_store_ratio',
       'promotion_depth', 'lag_1_promotion_depth', 'lag_1_ptc',
       'uch_level_g_cat', 'promo_ind'],
      dtype='object')

In [ ]:
df[df.promotion_depth.isnull()]

,uch_level_g,basecode,suggested_ptc,planned_ptc,holiady_ind,curr_yr_sales_dollars,curr_yr_sales_units,ptc,selling_store_count,actual_str_cnt,selling_store_ratio,promotion_depth,lag_1_promotion_depth,lag_1_ptc,uch_level_g_cat,promo_ind
fscl_wk_end_dt,,,,,,,,,,,,,,,,


In [ ]:
X = df[['basecode', 'suggested_ptc', 'planned_ptc',
       'holiady_ind', 'selling_store_count', 'actual_str_cnt', 'selling_store_ratio',
       'promotion_depth', 'lag_1_promotion_depth', 'lag_1_ptc',
       'uch_level_g_cat', 'promo_ind']][(df.uch_level_g_cat == 24) & (df.basecode == 681008959100)]
y = df[(df.uch_level_g_cat == 24) & (df.basecode == 681008959100)]['curr_yr_sales_units'].apply(lambda x: 1 if x <= df['curr_yr_sales_units'].median() else 0)
X = X.reset_index().drop('fscl_wk_end_dt',1)
y= y.reset_index().drop('fscl_wk_end_dt',1)

In [ ]:
y

,curr_yr_sales_units
0,0
1,0
2,0
3,0
4,0
...,...
132342,1
132343,1
132344,1
132345,1


In [ ]:
pipe = make_pipeline(StandardScaler(),
                     ColumnSelector(cols=('basecode', 'suggested_ptc', 'planned_ptc',
       'holiady_ind', 'selling_store_count', 'actual_str_cnt', 'selling_store_ratio',
       'promotion_depth', 'lag_1_promotion_depth', 'lag_1_ptc',
       'uch_level_g_cat', 'promo_ind')),
                     KNeighborsClassifier())

pipe.fit(X, y)
pipe.score(X, y)

IndexError: ignored

In [ ]:
from itertools import combinations

all_comb = []
for size in range(1, len(X.columns)):
    all_comb += list(combinations(range(X.shape[1]), r=size))
print(all_comb)

[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (7, 8), (7, 9), (7, 10), (7, 11), (8, 9), (8, 10), (8, 11), (9, 10), (9, 11), (10, 11), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7), (0, 1, 8), (0, 1, 9), (0, 1, 10), (0, 1, 11), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 2, 6), (0, 2, 7), (0, 2, 8), (0, 2, 9), (0, 2, 10), (0, 2, 11), (0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 3, 9), (0, 3, 10), (0, 3, 11), (0, 4, 5), (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 4, 9), (0, 4, 10), (0, 4

In [ ]:
from mlxtend.feature_selection import ColumnSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(StandardScaler(),
                     ColumnSelector(),
                     KNeighborsClassifier())

param_grid = {'columnselector__cols': all_comb,
              'kneighborsclassifier__n_neighbors': list(range(1, 11))}

grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
grid.fit(X, y)
print('Best parameters:', grid.best_params_)
print('Best performance:', grid.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



Best parameters: {'columnselector__cols': (0,), 'kneighborsclassifier__n_neighbors': 1}
Best performance: 1.0


/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:354: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [ ]:
print('Best parameters:', grid.best_params_)
print('Best performance:', grid.best_score_)

Best parameters: {'columnselector__cols': (0,), 'kneighborsclassifier__n_neighbors': 1}
Best performance: 1.0


In [ ]:
!pip install featuretools

,basecode,suggested_ptc,planned_ptc,holiady_ind,selling_store_count,actual_str_cnt,selling_store_ratio,promotion_depth,lag_1_promotion_depth,lag_1_ptc,uch_level_g_cat,promo_ind
0,681000111700,4.32,4.99,0,440,588,0.748299,1.155093,1.155093,4.99,0,0
1,681000112500,4.32,4.99,0,455,588,0.773810,1.155093,1.155093,4.99,0,0
2,681000145500,3.04,4.49,0,429,588,0.729592,1.476974,0.654605,1.99,0,0
3,681000145900,3.04,4.49,0,419,588,0.712585,1.476974,0.654605,1.99,0,0
4,681000146500,3.04,4.49,0,358,588,0.608844,1.476974,0.654605,1.99,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
132342,681009006400,5.76,5.99,0,1,963,0.001038,1.039931,1.039931,5.99,24,1
132343,681009006400,5.76,6.46,0,10,963,0.010384,1.121528,0.887153,5.11,24,0
132344,681009006600,4.32,5.40,0,12,963,0.012461,1.250000,1.233796,5.33,24,0
132345,681009006600,4.32,4.49,0,3,963,0.003115,1.039352,1.039352,4.49,24,1
